get NPP from MYD MOD17AHGF

year: 2001 - 2020

Point: Japanese 4rd time Mesh 0.5 km centroid

## NNP Extraction
Net Primary Productivity

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from osgeo import gdal
import os
import glob
import rasterio

In [2]:
#year = 2015
aimFolder = "F:/17_Article/01_Data/02_NetPrimaryProductivity"
#os.mkdir(aimFolder + "\\temp")

In [35]:
def mergeAndResampleNppRaster(year, aimFolder):
    #"""
    #Merge MOD and MYD,
    #Resample the raster with 0.004 degrees
    #"""
    rasterFile = glob.glob("F:/17_Article/01_Data/02_NetPrimaryProductivity/Net_PP_Yearly_500m_v6/Npp/*"+str(year) +"_001.tif")

    if year > 2002:
        tifflayer_0 = gdal.Open(rasterFile[0], gdal.GA_ReadOnly)
        tifflayer_1 = gdal.Open(rasterFile[1], gdal.GA_ReadOnly)

        nband = 1
        geotransform = tifflayer_0.GetGeoTransform()
        spatialreference = tifflayer_0.GetProjection()

        tifflayer_0_array = tifflayer_0.ReadAsArray()
        tifflayer_0_array = np.array(tifflayer_0_array)
        tifflayer_0_array = tifflayer_0_array.astype(float)
        tifflayer_0_array[tifflayer_0_array == 65535] = np.nan
        tifflayer_0_array[tifflayer_0_array > 32760] = 0

        tifflayer_1_array = tifflayer_1.ReadAsArray()
        tifflayer_1_array = np.array(tifflayer_1_array)
        tifflayer_1_array = tifflayer_1_array.astype(float)
        tifflayer_1_array[tifflayer_1_array == 65535] = np.nan
        tifflayer_1_array[tifflayer_1_array > 32760] = 0

        tifflayer_double_array = np.array([tifflayer_0_array, tifflayer_1_array])
        tifflayer_mean_array = np.nanmean(tifflayer_double_array, axis = 0)
    else:
        tifflayer_0 = gdal.Open(rasterFile[0], gdal.GA_ReadOnly)

        nband = 1
        geotransform = tifflayer_0.GetGeoTransform()
        spatialreference = tifflayer_0.GetProjection()

        tifflayer_0_array = tifflayer_0.ReadAsArray()
        tifflayer_0_array = np.array(tifflayer_0_array)
        tifflayer_0_array = tifflayer_0_array.astype(float)
        tifflayer_0_array[tifflayer_0_array == 65535] = np.nan
        tifflayer_0_array[tifflayer_0_array > 32760] = 0
        tifflayer_mean_array = tifflayer_0_array 


    ncol = tifflayer_mean_array.shape[1]
    nrow = tifflayer_mean_array.shape[0]

    temp_file = aimFolder + "/temp/temp_file_" + str(year) + ".tif"

    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(temp_file, ncol, nrow, nband, gdal.GDT_Float32)
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(tifflayer_mean_array)
    dst_dataset = None

    original_temp_raster = gdal.Open(temp_file, gdal.GA_ReadOnly)
    raster_rprj = gdal.Warp(aimFolder + "/temp/temp_file_" + str(year) + "0004.tif", original_temp_raster, dstSRS = "EPSG:4326", 
                            xRes = 0.004, yRes = -0.004, resampleAlg = "average", outputBounds = [120, 20, 154, 46])
    original_temp_raster = None
    raster_rprj = None


In [36]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

#Parallel(n_jobs=2)(delayed(mergeAndResampleNppRaster)(int(year), aimFolder) for year in np.linspace(2004, 2020, 17))
for year in np.linspace(2001, 2020, 20):
    mergeAndResampleNppRaster(int(year), aimFolder)

C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_20320\1240927100.py:29: RuntimeWarning: Mean of empty slice
  tifflayer_mean_array = np.nanmean(tifflayer_double_array, axis = 0)


In [10]:
np.linspace(2004, 2020, 17)

array([2004., 2005., 2006., 2007., 2008., 2009., 2010., 2011., 2012.,
       2013., 2014., 2015., 2016., 2017., 2018., 2019., 2020.])

In [15]:
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")

In [73]:
#os.remove(temp_file)
#os.remove(aimFolder + "/temp/temp_file_0008.tif")

#os.rmdir(aimFolder + "\\temp")

In [15]:
def nppExtraction(year, aimFolder, coords_extration):
    # this is to extract npp of each eyar of each mesh point
    rasterFile = rasterio.open(aimFolder + "/temp/temp_file_" + str(year) + "0004.tif")
    rasterArray = rasterFile.read(1)

    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        try:
            valueArray.append(rasterArray[row, col])
        except:
            valueArray.append(np.nan)
        
    valueArray = np.array(valueArray)
    coords_extration['NPP'] = valueArray

    rasterFile = None
    return coords_extration

In [16]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

aimFolder = "F:/17_Article/01_Data/02_NetPrimaryProductivity"
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")
nppResult = Parallel(n_jobs=4)(delayed(nppExtraction)(int(year), aimFolder, coords_extration) for year in np.linspace(2001, 2020, 20))

In [37]:
nppResult_2001 = nppResult[0]
nppResult_2002 = nppResult[1]
nppResult_2003 = nppResult[2]
nppResult_2004 = nppResult[3]
nppResult_2005 = nppResult[4]
nppResult_2006 = nppResult[5]
nppResult_2007 = nppResult[6]
nppResult_2008 = nppResult[7]
nppResult_2009 = nppResult[8]
nppResult_2010 = nppResult[9]
nppResult_2011 = nppResult[10]
nppResult_2012 = nppResult[11]
nppResult_2013 = nppResult[12]
nppResult_2014 = nppResult[13]
nppResult_2015 = nppResult[14]
nppResult_2016 = nppResult[15]
nppResult_2017 = nppResult[16]
nppResult_2018 = nppResult[17]
nppResult_2019 = nppResult[18]
nppResult_2020 = nppResult[19]

In [38]:
nppResult_2001['year'] = 2001
nppResult_2002['year'] = 2002
nppResult_2003['year'] = 2003
nppResult_2004['year'] = 2004
nppResult_2005['year'] = 2005
nppResult_2006['year'] = 2006
nppResult_2007['year'] = 2007
nppResult_2008['year'] = 2008
nppResult_2009['year'] = 2009
nppResult_2010['year'] = 2010
nppResult_2011['year'] = 2011
nppResult_2012['year'] = 2012
nppResult_2013['year'] = 2013
nppResult_2014['year'] = 2014
nppResult_2015['year'] = 2015
nppResult_2016['year'] = 2016
nppResult_2017['year'] = 2017
nppResult_2018['year'] = 2018
nppResult_2019['year'] = 2019
nppResult_2020['year'] = 2020

In [39]:
nppResult_2001 = pd.DataFrame(nppResult_2001.drop(columns='geometry'))
nppResult_2002 = pd.DataFrame(nppResult_2002.drop(columns='geometry'))
nppResult_2003 = pd.DataFrame(nppResult_2003.drop(columns='geometry'))
nppResult_2004 = pd.DataFrame(nppResult_2004.drop(columns='geometry'))
nppResult_2005 = pd.DataFrame(nppResult_2005.drop(columns='geometry'))
nppResult_2006 = pd.DataFrame(nppResult_2006.drop(columns='geometry'))
nppResult_2007 = pd.DataFrame(nppResult_2007.drop(columns='geometry'))
nppResult_2008 = pd.DataFrame(nppResult_2008.drop(columns='geometry'))
nppResult_2009 = pd.DataFrame(nppResult_2009.drop(columns='geometry'))
nppResult_2010 = pd.DataFrame(nppResult_2010.drop(columns='geometry'))
nppResult_2011 = pd.DataFrame(nppResult_2011.drop(columns='geometry'))
nppResult_2012 = pd.DataFrame(nppResult_2012.drop(columns='geometry'))
nppResult_2013 = pd.DataFrame(nppResult_2013.drop(columns='geometry'))
nppResult_2014 = pd.DataFrame(nppResult_2014.drop(columns='geometry'))
nppResult_2015 = pd.DataFrame(nppResult_2015.drop(columns='geometry'))
nppResult_2016 = pd.DataFrame(nppResult_2016.drop(columns='geometry'))
nppResult_2017 = pd.DataFrame(nppResult_2017.drop(columns='geometry'))
nppResult_2018 = pd.DataFrame(nppResult_2018.drop(columns='geometry'))
nppResult_2019 = pd.DataFrame(nppResult_2019.drop(columns='geometry'))
nppResult_2020 = pd.DataFrame(nppResult_2020.drop(columns='geometry'))

In [41]:
nppResult_2001.columns

Index(['G04c_001', 'NPP', 'year'], dtype='object')

In [21]:
nppResult_2001.columns = ['G04c_001', 'NPP', 'year']
nppResult_2002.columns = ['G04c_001', 'NPP', 'year']
nppResult_2003.columns = ['G04c_001', 'NPP', 'year']
nppResult_2004.columns = ['G04c_001', 'NPP', 'year']
nppResult_2005.columns = ['G04c_001', 'NPP', 'year']
nppResult_2006.columns = ['G04c_001', 'NPP', 'year']
nppResult_2007.columns = ['G04c_001', 'NPP', 'year']
nppResult_2008.columns = ['G04c_001', 'NPP', 'year']
nppResult_2009.columns = ['G04c_001', 'NPP', 'year']
nppResult_2010.columns = ['G04c_001', 'NPP', 'year']
nppResult_2011.columns = ['G04c_001', 'NPP', 'year']
nppResult_2012.columns = ['G04c_001', 'NPP', 'year']
nppResult_2013.columns = ['G04c_001', 'NPP', 'year']
nppResult_2014.columns = ['G04c_001', 'NPP', 'year']
nppResult_2015.columns = ['G04c_001', 'NPP', 'year']
nppResult_2016.columns = ['G04c_001', 'NPP', 'year']
nppResult_2017.columns = ['G04c_001', 'NPP', 'year']
nppResult_2018.columns = ['G04c_001', 'NPP', 'year']
nppResult_2019.columns = ['G04c_001', 'NPP', 'year']
nppResult_2020.columns = ['G04c_001', 'NPP', 'year']

In [42]:
print(nppResult_2001[pd.isna(nppResult_2001["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2002["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2003["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2004["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2005["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2006["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2007["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2008["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2009["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2010["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2011["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2012["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2013["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2014["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2015["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2016["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2017["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2018["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2019["NPP"])].count())
print(nppResult_2001[pd.isna(nppResult_2020["NPP"])].count())

G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16
NPP          0
year        16
dtype: int64
G04c_001    16

In [43]:
nppResult_2001[pd.isna(nppResult_2001["NPP"])].shape
            

(16, 3)

In [44]:
longNppDF = pd.concat(
    [nppResult_2001, nppResult_2002, nppResult_2003, nppResult_2004, nppResult_2005, nppResult_2006, nppResult_2007, nppResult_2008,
    nppResult_2009, nppResult_2010, nppResult_2011, nppResult_2012, nppResult_2013, nppResult_2014, nppResult_2015, nppResult_2016,
    nppResult_2017, nppResult_2018, nppResult_2019, nppResult_2020]
)

In [45]:
del nppResult_2001
del nppResult_2002
del nppResult_2003
del nppResult_2004
del nppResult_2005
del nppResult_2006
del nppResult_2007
del nppResult_2008
del nppResult_2009
del nppResult_2010
del nppResult_2011
del nppResult_2012
del nppResult_2013
del nppResult_2014
del nppResult_2015
del nppResult_2016
del nppResult_2017
del nppResult_2018
del nppResult_2019
del nppResult_2020

import gc
gc.collect()

1132

In [46]:
longNppDF = longNppDF.set_index(['G04c_001','year'])
longNppDF.mean()

NPP    6818.637622
dtype: float64

In [47]:
fileList = glob.glob(aimFolder + "/temp/*.tif")
for filename in fileList:
    os.remove(filename)

os.rmdir(aimFolder + "\\temp")

PermissionError: [WinError 5] 拒绝访问。: 'F:/17_Article/01_Data/02_NetPrimaryProductivity/temp\\temp_file_20160004.tif'

In [88]:
longNppDF.head()

,,NPP
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


## NTL extraction
Night Time Light

In [48]:
import os
import zipfile
import glob
import rasterio
from osgeo import gdal
import geopandas as gpd
import numpy as np

In [49]:
aimFolder = "F:\\17_Article\\01_Data\\03_NTL_VIIRS"
#os.mkdir()

In [50]:
def ntlExtraction(year, aimFolder, coords_extration):
    #This funciton is try to extract ntl
    #filename = aimFolder + "\\" + str(year) + ".zip"
    #with zipfile.ZipFile(filename, "r") as zip_ref:
    #    zip_ref.extractall(aimFolder + "\\temp")
    aimRaster = glob.glob(aimFolder + "\\temp\\*" + str(year) + ".tif")
    aimRaster = aimRaster[0]
    rasterFile = rasterio.open(aimRaster)
    rasterArray = rasterFile.read(1)
    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
    
    valueArray = np.array(valueArray)
    coords_extration['NTL'] = valueArray
    return coords_extration

In [51]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")
ntlResult = Parallel(n_jobs=2)(delayed(ntlExtraction)(int(year), aimFolder, coords_extration) for year in np.linspace(2002, 2020, 19))

In [52]:
ntlResult_2001 = ntlExtraction(int(2001), aimFolder, coords_extration)

In [53]:
ntlResult_2002 = ntlResult[0]
ntlResult_2003 = ntlResult[1]
ntlResult_2004 = ntlResult[2]
ntlResult_2005 = ntlResult[3]
ntlResult_2006 = ntlResult[4]
ntlResult_2007 = ntlResult[5]
ntlResult_2008 = ntlResult[6]
ntlResult_2009 = ntlResult[7]
ntlResult_2010 = ntlResult[8]
ntlResult_2011 = ntlResult[9]
ntlResult_2012 = ntlResult[10]
ntlResult_2013 = ntlResult[11]
ntlResult_2014 = ntlResult[12]
ntlResult_2015 = ntlResult[13]
ntlResult_2016 = ntlResult[14]
ntlResult_2017 = ntlResult[15]
ntlResult_2018 = ntlResult[16]
ntlResult_2019 = ntlResult[17]
ntlResult_2020 = ntlResult[18]

In [54]:
ntlResult_2001['year'] = 2001
ntlResult_2002['year'] = 2002
ntlResult_2003['year'] = 2003
ntlResult_2004['year'] = 2004
ntlResult_2005['year'] = 2005
ntlResult_2006['year'] = 2006
ntlResult_2007['year'] = 2007
ntlResult_2008['year'] = 2008
ntlResult_2009['year'] = 2009
ntlResult_2010['year'] = 2010
ntlResult_2011['year'] = 2011
ntlResult_2012['year'] = 2012
ntlResult_2013['year'] = 2013
ntlResult_2014['year'] = 2014
ntlResult_2015['year'] = 2015
ntlResult_2016['year'] = 2016
ntlResult_2017['year'] = 2017
ntlResult_2018['year'] = 2018
ntlResult_2019['year'] = 2019
ntlResult_2020['year'] = 2020

In [55]:
ntlResult_2001 = pd.DataFrame(ntlResult_2001.drop(columns='geometry'))
ntlResult_2002 = pd.DataFrame(ntlResult_2002.drop(columns='geometry'))
ntlResult_2003 = pd.DataFrame(ntlResult_2003.drop(columns='geometry'))
ntlResult_2004 = pd.DataFrame(ntlResult_2004.drop(columns='geometry'))
ntlResult_2005 = pd.DataFrame(ntlResult_2005.drop(columns='geometry'))
ntlResult_2006 = pd.DataFrame(ntlResult_2006.drop(columns='geometry'))
ntlResult_2007 = pd.DataFrame(ntlResult_2007.drop(columns='geometry'))
ntlResult_2008 = pd.DataFrame(ntlResult_2008.drop(columns='geometry'))
ntlResult_2009 = pd.DataFrame(ntlResult_2009.drop(columns='geometry'))
ntlResult_2010 = pd.DataFrame(ntlResult_2010.drop(columns='geometry'))
ntlResult_2011 = pd.DataFrame(ntlResult_2011.drop(columns='geometry'))
ntlResult_2012 = pd.DataFrame(ntlResult_2012.drop(columns='geometry'))
ntlResult_2013 = pd.DataFrame(ntlResult_2013.drop(columns='geometry'))
ntlResult_2014 = pd.DataFrame(ntlResult_2014.drop(columns='geometry'))
ntlResult_2015 = pd.DataFrame(ntlResult_2015.drop(columns='geometry'))
ntlResult_2016 = pd.DataFrame(ntlResult_2016.drop(columns='geometry'))
ntlResult_2017 = pd.DataFrame(ntlResult_2017.drop(columns='geometry'))
ntlResult_2018 = pd.DataFrame(ntlResult_2018.drop(columns='geometry'))
ntlResult_2019 = pd.DataFrame(ntlResult_2019.drop(columns='geometry'))
ntlResult_2020 = pd.DataFrame(ntlResult_2020.drop(columns='geometry'))

In [56]:
ntlResult_2001.columns

Index(['G04c_001', 'NTL', 'year'], dtype='object')

In [57]:
ntlResult_2001.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2002.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2003.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2004.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2005.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2006.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2007.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2008.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2009.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2010.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2011.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2012.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2013.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2014.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2015.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2016.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2017.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2018.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2019.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2020.columns = ['G04c_001', 'NTL', 'year']

In [58]:
ntlResult_2020.NTL[ntlResult_2020["NTL"]<0] = 0
ntlResult_2020["NTL"].mean()
ntlResult_2019.NTL[ntlResult_2019["NTL"]<0] = 0
ntlResult_2019["NTL"].mean()


C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_20320\1752473119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ntlResult_2020.NTL[ntlResult_2020["NTL"]<0] = 0
C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_20320\1752473119.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ntlResult_2019.NTL[ntlResult_2019["NTL"]<0] = 0


1.1114916

In [59]:
print(ntlResult_2001["NTL"].mean())
print(ntlResult_2002["NTL"].mean())
print(ntlResult_2003["NTL"].mean())
print(ntlResult_2004["NTL"].mean())
print(ntlResult_2005["NTL"].mean())
print(ntlResult_2006["NTL"].mean())
print(ntlResult_2007["NTL"].mean())
print(ntlResult_2008["NTL"].mean())
print(ntlResult_2009["NTL"].mean())
print(ntlResult_2010["NTL"].mean())
print(ntlResult_2011["NTL"].mean())
print(ntlResult_2012["NTL"].mean())
print(ntlResult_2013["NTL"].mean())
print(ntlResult_2014["NTL"].mean())
print(ntlResult_2015["NTL"].mean())
print(ntlResult_2016["NTL"].mean())
print(ntlResult_2017["NTL"].mean())
print(ntlResult_2018["NTL"].mean())
print(ntlResult_2019["NTL"].mean())
print(ntlResult_2020["NTL"].mean())

1.6480222
1.6131924
1.4684126
1.2629389
1.5879596
1.575272
1.6215699
1.4386377
1.3462512
1.0992323
1.3615805
1.3009588
1.061045
1.0638536
1.0337858
1.0159932
1.1423855
1.1576278
1.1114916
1.0489889


In [60]:
ntlResult_2001.head()

,G04c_001,NTL,year
0,303660001,0.0,2001
1,303660002,0.0,2001
2,303660003,0.0,2001
3,303660004,0.0,2001
4,303660011,0.0,2001


In [62]:
ntlResult_2002[pd.isna(ntlResult_2002["NTL"])].head()

,G04c_001,NTL,year


In [64]:
longNtlDF[pd.isna(longNtlDF["NTL"])].head()

,G04c_001,NTL,year


In [63]:
longNtlDF = pd.concat(
    [ntlResult_2001, ntlResult_2002, ntlResult_2003, ntlResult_2004, ntlResult_2005, ntlResult_2006, ntlResult_2007, ntlResult_2008,
    ntlResult_2009, ntlResult_2010, ntlResult_2011, ntlResult_2012, ntlResult_2013, ntlResult_2014, ntlResult_2015, ntlResult_2016,
    ntlResult_2017, ntlResult_2018, ntlResult_2019, ntlResult_2020]
)

In [65]:
del ntlResult_2001
del ntlResult_2002
del ntlResult_2003
del ntlResult_2004
del ntlResult_2005
del ntlResult_2006
del ntlResult_2007
del ntlResult_2008
del ntlResult_2009
del ntlResult_2010
del ntlResult_2011
del ntlResult_2012
del ntlResult_2013
del ntlResult_2014
del ntlResult_2015
del ntlResult_2016
del ntlResult_2017
del ntlResult_2018
del ntlResult_2019
del ntlResult_2020

import gc
gc.collect()

178

In [66]:
longNtlDF = longNtlDF.set_index(['G04c_001','year'])
longNtlDF.head()

,,NTL
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


In [67]:

longNtlDF.mean()

NTL    1.297961
dtype: float32

In [68]:
del ntlResult
gc.collect()

77

In [69]:
fileList = glob.glob(aimFolder + "\\temp\\*")
#fileList = fileList[1:]
for filename in fileList:
    os.remove(filename)

#os.rmdir(aimFolder + "\\temp")

In [70]:
os.rmdir(aimFolder + "\\temp")

In [89]:
longNtlDF.head()

,,NTL
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


# Temperature Extraction
Surf_Temp_8Days_1Km (M*D11A2)

In [73]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [74]:
import os
import glob
from osgeo import gdal
import numpy as np
import gc
import geopandas as gpd
import rasterio
import math

In [75]:
aimFolder = "F:\\17_Article\\01_Data\\08_Temperature\\Surf_Temp_8Days_1Km_v6"
#os.mkdir(aimFolder + "\\temp")
#year = 2015

In [107]:
def dayTimeTemperatureRaster(year, aimFolder):
### Day time Temp
    dayTimeTemperatureFileList = glob.glob(aimFolder + "\\LST_Day_1km\\M*" + str(year) + "*.tif")

    dayTimeTemperatureRasterList = []
    for dayTimeTemperatureFile in dayTimeTemperatureFileList:
        dayTimeTemperatureRasterRaw = gdal.Open(dayTimeTemperatureFile, gdal.GA_ReadOnly)
        dayTimeTemperatureRaster = dayTimeTemperatureRasterRaw.ReadAsArray()
        dayTimeTemperatureRasterRaw = None
        dayTimeTemperatureRaster = dayTimeTemperatureRaster.astype("float")
        dayTimeTemperatureRaster[dayTimeTemperatureRaster == 0] = np.nan
        dayTimeTemperatureRaster = dayTimeTemperatureRaster * 0.02 - 273.16
        dayTimeTemperatureRasterList.append(dayTimeTemperatureRaster)

    meanDayTimeTemperatureRaster = np.nanmean(dayTimeTemperatureRasterList, axis = 0)
    stdDayTimeTemperatureRaster = np.nanstd(dayTimeTemperatureRasterList, axis = 0)
    dayTimeTemperatureRasterList = None
    gc.collect()

    src_dataset = gdal.Open(dayTimeTemperatureFileList[0], gdal.GA_ReadOnly)
    geotransform = src_dataset.GetGeoTransform()
    spatialreference = src_dataset.GetProjection()
    ncol = meanDayTimeTemperatureRaster.shape[1]
    nrow = meanDayTimeTemperatureRaster.shape[0]
    nband = 1
    src_dataset = None

    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(aimFolder + "\\temp\\" + str(year) + "_daytime_mean.tif", ncol, nrow, nband, gdal.GDT_Float32) 
    #### ^^^^^ Change this line
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(meanDayTimeTemperatureRaster)
    dst_dataset = None

    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(aimFolder + "\\temp\\" + str(year) + "_daytime_std.tif", ncol, nrow, nband, gdal.GDT_Float32) 
    #### ^^^^^ Change this line
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(stdDayTimeTemperatureRaster)
    dst_dataset = None

    ### resample and reproject
    raster_rprj = gdal.Warp(aimFolder + "\\temp\\" + str(year) + "_daytime_mean_re.tif", 
                            aimFolder + "\\temp\\" + str(year) + "_daytime_mean.tif", dstSRS = "EPSG:4326",
                            xRes = 0.008, yRes = 0.008, resampleAlg = "average", srcNodata = math.nan)
    raster_rprj = None

    raster_rprj = gdal.Warp(aimFolder + "\\temp\\" + str(year) + "_daytime_std_re.tif", 
                            aimFolder + "\\temp\\" + str(year) + "_daytime_std.tif", dstSRS = "EPSG:4326",
                            xRes = 0.008, yRes = 0.008, resampleAlg = "average", srcNodata = math.nan)
    raster_rprj = None


In [113]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

#Parallel(n_jobs=2)(delayed(dayTimeTemperatureRaster)(int(year), aimFolder) for year in np.linspace(2001, 2020, 20))
#parallel unavailable

for year in np.linspace(2003, 2020, 18):
    dayTimeTemperatureRaster(int(year), aimFolder)

C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_12728\1949070710.py:15: RuntimeWarning: Mean of empty slice
  meanDayTimeTemperatureRaster = np.nanmean(dayTimeTemperatureRasterList, axis = 0)
d:\Anaconda3\envs\Usegdal\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [115]:
gc.collect()

154

In [118]:
def nightTimeTemperatureRaster(year, aimFolder):
    ### Night time Temp
    nightTimeTemperatureFileList = glob.glob(aimFolder + "\\LST_Night_1km\\M*" + str(year) + "*.tif")

    nightTimeTemperatureRasterList = []
    for nightTimeTemperatureFile in nightTimeTemperatureFileList:
        nightTimeTemperatureRasterRaw = gdal.Open(nightTimeTemperatureFile, gdal.GA_ReadOnly)
        nightTimeTemperatureRaster = nightTimeTemperatureRasterRaw.ReadAsArray()
        nightTimeTemperatureRasterRaw = None
        nightTimeTemperatureRaster = nightTimeTemperatureRaster.astype("float")
        nightTimeTemperatureRaster[nightTimeTemperatureRaster == 0] = np.nan
        nightTimeTemperatureRaster = nightTimeTemperatureRaster * 0.02 - 273.16
        nightTimeTemperatureRasterList.append(nightTimeTemperatureRaster)

    meanNightTimeTemperatureRaster = np.nanmean(nightTimeTemperatureRasterList, axis = 0)
    stdNightTimeTemperatureRaster = np.nanstd(nightTimeTemperatureRasterList, axis = 0)
    nightTimeTemperatureRasterList = None
    gc.collect()

    src_dataset = gdal.Open(nightTimeTemperatureFileList[0], gdal.GA_ReadOnly)
    geotransform = src_dataset.GetGeoTransform()
    spatialreference = src_dataset.GetProjection()
    ncol = meanNightTimeTemperatureRaster.shape[1]
    nrow = meanNightTimeTemperatureRaster.shape[0]
    nband = 1
    src_dataset = None

    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(aimFolder + "\\temp\\" + str(year) + "_nighttime_mean.tif", ncol, nrow, nband, gdal.GDT_Float32) 
    #### ^^^^^ Change this line
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(meanNightTimeTemperatureRaster)
    dst_dataset = None

    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(aimFolder + "\\temp\\" + str(year) + "_nighttime_std.tif", ncol, nrow, nband, gdal.GDT_Float32) 
    #### ^^^^^ Change this line
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(stdNightTimeTemperatureRaster)
    dst_dataset = None

    ### resample and reproject
    raster_rprj = gdal.Warp(aimFolder + "\\temp\\" + str(year) + "_nighttime_mean_re.tif", 
                            aimFolder + "\\temp\\" + str(year) + "_nighttime_mean.tif", dstSRS = "EPSG:4326",
                            xRes = 0.008, yRes = 0.008, resampleAlg = "average", srcNodata = math.nan)
    raster_rprj = None

    raster_rprj = gdal.Warp(aimFolder + "\\temp\\" + str(year) + "_nighttime_std_re.tif", 
                            aimFolder + "\\temp\\" + str(year) + "_nighttime_std.tif", dstSRS = "EPSG:4326",
                            xRes = 0.008, yRes = 0.008, resampleAlg = "average", srcNodata = math.nan)
    raster_rprj = None

In [119]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

for year in np.linspace(2001, 2020, 20):
    nightTimeTemperatureRaster(int(year), aimFolder)

C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_12728\342181939.py:15: RuntimeWarning: Mean of empty slice
  meanNightTimeTemperatureRaster = np.nanmean(nightTimeTemperatureRasterList, axis = 0)


In [76]:
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")

In [77]:
def coordExtractionFromRaster(RasterName, GeoPandasDataFrame, NewColumnName):
    rasterFile = rasterio.open(RasterName)
    coords_extration = GeoPandasDataFrame.copy()
    rasterArray = rasterFile.read(1)
    
    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
        
    valueArray = np.array(valueArray)
    coords_extration[NewColumnName] = valueArray
    rasterFile = None
    return coords_extration

In [135]:

from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

dayTimeTempMeanResult = Parallel(n_jobs=8)(delayed(coordExtractionFromRaster)(aimFolder + "\\temp\\" + str(int(year)) + "_daytime_mean_re.tif", coords_extration, 'dayTimeMeanTemp') for year in np.linspace(2001, 2020, 20))

In [136]:
dayTimeTempMeanResult[0].columns

Index(['G04c_001', 'geometry', 'dayTimeMeanTemp'], dtype='object')

In [137]:
def mergeParallelDf(DF):
    singleDF_0 = DF[0]
    singleDF_1 = DF[1]
    singleDF_2 = DF[2]
    singleDF_3 = DF[3]
    singleDF_4 = DF[4]
    singleDF_5 = DF[5]
    singleDF_6 = DF[6]
    singleDF_7 = DF[7]
    singleDF_8 = DF[8]
    singleDF_9 = DF[9]
    singleDF_10 = DF[10]
    singleDF_11 = DF[11]
    singleDF_12 = DF[12]
    singleDF_13 = DF[13]
    singleDF_14 = DF[14]
    singleDF_15 = DF[15]
    singleDF_16 = DF[16]
    singleDF_17 = DF[17]
    singleDF_18 = DF[18]
    singleDF_19 = DF[19]

    singleDF_0['year'] = 2001
    singleDF_1['year'] = 2002 
    singleDF_2['year'] = 2003 
    singleDF_3['year'] = 2004 
    singleDF_4['year'] = 2005 
    singleDF_5['year'] = 2006 
    singleDF_6['year'] = 2007 
    singleDF_7['year'] = 2008 
    singleDF_8['year'] = 2009 
    singleDF_9['year'] = 2010 
    singleDF_10['year'] = 2011
    singleDF_11['year'] = 2012 
    singleDF_12['year'] = 2013 
    singleDF_13['year'] = 2014 
    singleDF_14['year'] = 2015 
    singleDF_15['year'] = 2016 
    singleDF_16['year'] = 2017 
    singleDF_17['year'] = 2018 
    singleDF_18['year'] = 2019 
    singleDF_19['year'] = 2020

    mergeDF = pd.concat([
        singleDF_0, singleDF_1, singleDF_2, singleDF_3, singleDF_4, singleDF_5, singleDF_6, singleDF_7, singleDF_8, singleDF_9,
        singleDF_10, singleDF_11, singleDF_12, singleDF_13, singleDF_14, singleDF_15, singleDF_16, singleDF_17, singleDF_18, singleDF_19
    ])

    return mergeDF


In [138]:
import pandas as pd
mergeDayTimeTempMeanResult = mergeParallelDf(dayTimeTempMeanResult)

In [139]:
mergeDayTimeTempMeanResult.columns

Index(['G04c_001', 'geometry', 'dayTimeMeanTemp', 'year'], dtype='object')

In [140]:
print(singleDF_0[pd.isna(singleDF_0['dayTimeMeanTemp'])].shape)
print(singleDF_1[pd.isna(singleDF_1['dayTimeMeanTemp'])].shape)
print(singleDF_2[pd.isna(singleDF_2['dayTimeMeanTemp'])].shape)
print(singleDF_3[pd.isna(singleDF_3['dayTimeMeanTemp'])].shape)
print(singleDF_4[pd.isna(singleDF_4['dayTimeMeanTemp'])].shape)
print(singleDF_5[pd.isna(singleDF_5['dayTimeMeanTemp'])].shape)
print(singleDF_6[pd.isna(singleDF_6['dayTimeMeanTemp'])].shape)
print(singleDF_7[pd.isna(singleDF_7['dayTimeMeanTemp'])].shape)
print(singleDF_8[pd.isna(singleDF_8['dayTimeMeanTemp'])].shape)
print(singleDF_9[pd.isna(singleDF_9['dayTimeMeanTemp'])].shape)

(26241, 4)
(26193, 4)
(26189, 4)
(26179, 4)
(26201, 4)
(26173, 4)
(26159, 4)
(26182, 4)
(26173, 4)
(26216, 4)


In [141]:
mergeDayTimeTempMeanResult = pd.DataFrame(mergeDayTimeTempMeanResult.drop(columns='geometry'))
mergeDayTimeTempMeanResult = mergeDayTimeTempMeanResult[['G04c_001', 'dayTimeMeanTemp', 'year']]
mergeDayTimeTempMeanResult = mergeDayTimeTempMeanResult.set_index(['G04c_001', 'year'])


In [142]:
mergeDayTimeTempMeanResult[pd.isna(mergeDayTimeTempMeanResult['dayTimeMeanTemp'])].shape[0]/20

26196.05

In [143]:
mergeDayTimeTempMeanResult['dayTimeMeanTemp'] = mergeDayTimeTempMeanResult['dayTimeMeanTemp'].fillna(0)

In [145]:
del dayTimeTempMeanResult
gc.collect()

NameError: name 'dayTimeTempMeanResult' is not defined

In [146]:
mergeDayTimeTempMeanResult.head()

,,dayTimeMeanTemp
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


In [147]:
dayTimeTempStdResult = Parallel(n_jobs=8)(delayed(coordExtractionFromRaster)(aimFolder + "\\temp\\" + str(int(year)) + "_daytime_std_re.tif", coords_extration, 'dayTimeStdTemp') for year in np.linspace(2001, 2020, 20))

In [148]:
mergeDayTimeTempStdResult = mergeParallelDf(dayTimeTempStdResult)
mergeDayTimeTempStdResult = pd.DataFrame(mergeDayTimeTempStdResult.drop(columns='geometry'))
mergeDayTimeTempStdResult = mergeDayTimeTempStdResult[['G04c_001', 'dayTimeStdTemp', 'year']]
mergeDayTimeTempStdResult = mergeDayTimeTempStdResult.set_index(['G04c_001', 'year'])

In [149]:
mergeDayTimeTempStdResult[pd.isna(mergeDayTimeTempStdResult['dayTimeStdTemp'])].shape

(523921, 1)

In [150]:
mergeDayTimeTempStdResult['dayTimeStdTemp'] = mergeDayTimeTempStdResult['dayTimeStdTemp'].fillna(0)

In [153]:
mergeDayTimeTempResult = pd.concat([mergeDayTimeTempMeanResult, mergeDayTimeTempStdResult], axis=1)

In [154]:
mergeDayTimeTempResult.head()

,,dayTimeMeanTemp,dayTimeStdTemp
G04c_001,year,,
303660001,2001,0.0,0.0
303660002,2001,0.0,0.0
303660003,2001,0.0,0.0
303660004,2001,0.0,0.0
303660011,2001,0.0,0.0


In [155]:
print(mergeDayTimeTempResult.shape)
print(mergeDayTimeTempMeanResult.shape)
print(mergeDayTimeTempStdResult.shape)

(31060480, 2)
(31060480, 1)
(31060480, 1)


In [156]:
del dayTimeTempStdResult
gc.collect()

763

In [157]:
del mergeDayTimeTempMeanResult
del mergeDayTimeTempStdResult

gc.collect()

64

In [158]:
nightTimeTempMeanResult = Parallel(n_jobs=8)(delayed(coordExtractionFromRaster)(aimFolder + "\\temp\\" + str(int(year)) + "_nighttime_Mean_re.tif", coords_extration, 'nightTimeMeanTemp') for year in np.linspace(2001, 2020, 20))

In [159]:
mergeNightTimeTempMeanResult = mergeParallelDf(nightTimeTempMeanResult)
mergeNightTimeTempMeanResult = pd.DataFrame(mergeNightTimeTempMeanResult.drop(columns='geometry'))
mergeNightTimeTempMeanResult = mergeNightTimeTempMeanResult[['G04c_001', 'nightTimeMeanTemp', 'year']]
mergeNightTimeTempMeanResult = mergeNightTimeTempMeanResult.set_index(['G04c_001', 'year'])

In [160]:
mergeNightTimeTempMeanResult[pd.isna(mergeNightTimeTempMeanResult['nightTimeMeanTemp'])].shape

(524238, 1)

In [161]:
mergeNightTimeTempMeanResult['nightTimeMeanTemp'] = mergeNightTimeTempMeanResult['nightTimeMeanTemp'].fillna(0)

In [162]:
mergeNightTimeTempMeanResult.head()

,,nightTimeMeanTemp
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


In [163]:
del nightTimeTempMeanResult
gc.collect()

746

In [164]:
nightTimeTempStdResult = Parallel(n_jobs=8)(delayed(coordExtractionFromRaster)(aimFolder + "\\temp\\" + str(int(year)) + "_nighttime_Std_re.tif", coords_extration, 'nightTimeStdTemp') for year in np.linspace(2001, 2020, 20))

In [165]:
mergeNightTimeTempStdResult = mergeParallelDf(nightTimeTempStdResult)
mergeNightTimeTempStdResult = pd.DataFrame(mergeNightTimeTempStdResult.drop(columns='geometry'))
mergeNightTimeTempStdResult = mergeNightTimeTempStdResult[['G04c_001', 'nightTimeStdTemp', 'year']]
mergeNightTimeTempStdResult = mergeNightTimeTempStdResult.set_index(['G04c_001', 'year'])

In [166]:
mergeNightTimeTempStdResult[pd.isna(mergeNightTimeTempStdResult['nightTimeStdTemp'])].shape

(524238, 1)

In [167]:
mergeNightTimeTempStdResult['nightTimeStdTemp'] = mergeNightTimeTempStdResult['nightTimeStdTemp'].fillna(0)

In [168]:
mergeNightTimeTempStdResult.head()

,,nightTimeStdTemp
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


In [169]:
del nightTimeTempStdResult
gc.collect()

919

In [170]:
mergeNightTimeTempResult = pd.concat([mergeNightTimeTempMeanResult, mergeNightTimeTempStdResult], axis=1)

In [171]:
mergeNightTimeTempResult.shape

(31060480, 2)

In [172]:
mergeNightTimeTempResult.head()

,,nightTimeMeanTemp,nightTimeStdTemp
G04c_001,year,,
303660001,2001,0.0,0.0
303660002,2001,0.0,0.0
303660003,2001,0.0,0.0
303660004,2001,0.0,0.0
303660011,2001,0.0,0.0


In [173]:
mergeTempResult = pd.concat([mergeDayTimeTempResult, mergeNightTimeTempResult], axis=1)

In [174]:
mergeTempResult.shape

(31060480, 4)

In [175]:
mergeTempResult.head()

,,dayTimeMeanTemp,dayTimeStdTemp,nightTimeMeanTemp,nightTimeStdTemp
G04c_001,year,,,,
303660001,2001,0.0,0.0,0.0,0.0
303660002,2001,0.0,0.0,0.0,0.0
303660003,2001,0.0,0.0,0.0,0.0
303660004,2001,0.0,0.0,0.0,0.0
303660011,2001,0.0,0.0,0.0,0.0


In [176]:
del mergeNightTimeTempMeanResult
del mergeNightTimeTempStdResult
gc.collect()

75

In [177]:
del mergeDayTimeTempResult
del mergeNightTimeTempResult
gc.collect()

25

In [178]:
mergeTempResult.head()

,,dayTimeMeanTemp,dayTimeStdTemp,nightTimeMeanTemp,nightTimeStdTemp
G04c_001,year,,,,
303660001,2001,0.0,0.0,0.0,0.0
303660002,2001,0.0,0.0,0.0,0.0
303660003,2001,0.0,0.0,0.0,0.0
303660004,2001,0.0,0.0,0.0,0.0
303660011,2001,0.0,0.0,0.0,0.0


In [197]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

# Precipitaiton

GPM IMERG Final Precipitation L3 1 month 0.1 degree x 0.1 degree V06 (GPM_3IMERGM)  
https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGM_06/summary?keywords=GPM_3IMERGM.06

In [2]:
import numpy as np
import os
from osgeo import gdal
import glob
import geopandas as gpd
import rasterio
from osgeo import osr

In [3]:
rawFileLocation = "F:/17_Article/01_Data/11_precipitation/"

#os.mkdir(rawFileLocation + "/temp")

In [208]:
def precipitationRasterMaking(year, rawFileLocation):
    rawFileList = glob.glob(rawFileLocation + "/3B-MO.MS.MRG.3IMERG." + str(year) + "*.HDF5")
    annualAveragePrecipitation = []
    for hdf5File in rawFileList:
        hdflayer = gdal.Open(hdf5File, gdal.GA_ReadOnly)
        print(hdf5File)
        #for line in hdflayer.GetSubDatasets(): print(line)
        subhdflayer = hdflayer.GetSubDatasets()[3][0]
        rlayer = gdal.Open(subhdflayer, gdal.GA_ReadOnly)
        rlayerArray = rlayer.ReadAsArray()
        rlayerArray[rlayerArray < 0] = np.nan
        annualAveragePrecipitation.append(rlayerArray)
        rlayerArray = None
    annualAveragePrecipitation = np.nanmean(annualAveragePrecipitation, axis = 0)
    print(np.nanmean(annualAveragePrecipitation))   
    annualAveragePrecipitation = annualAveragePrecipitation.transpose()
    annualAveragePrecipitation = annualAveragePrecipitation[::-1,:]
    outputAnnualRasterName = rawFileLocation + "/temp/" + str(year) + "_precipitation_m.tif"
    ncol = annualAveragePrecipitation.shape[1]
    nrow = annualAveragePrecipitation.shape[0]
    nband = 1
    geotransform = (-180.0, 0.1, 0.0, 90.0, 0.0, -0.1)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    spatialreference = srs.ExportToWkt()
    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(outputAnnualRasterName, ncol, nrow, nband, gdal.GDT_Float32)
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(annualAveragePrecipitation)
    dst_dataset = None


In [210]:
from joblib import Parallel, delayed
#Parallel(n_jobs=1)(delayed(precipitationRasterMaking)(int(year), rawFileLocation) for year in np.linspace(2001, 2020, 20))
for year in np.linspace(2001, 2020, 20): 
    precipitationRasterMaking(int(year), rawFileLocation)

F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010101-S000000-E235959.01.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010201-S000000-E235959.02.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010301-S000000-E235959.03.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010401-S000000-E235959.04.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010501-S000000-E235959.05.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010601-S000000-E235959.06.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010701-S000000-E235959.07.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010801-S000000-E235959.08.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20010901-S000000-E235959.09.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20011001-S000000-E235959.10.V06B.HDF5
F:/17_Article/01_Data/11_preci

C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_20320\1382657533.py:14: RuntimeWarning: Mean of empty slice
  annualAveragePrecipitation = np.nanmean(annualAveragePrecipitation, axis = 0)


0.12284192
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020101-S000000-E235959.01.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020201-S000000-E235959.02.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020301-S000000-E235959.03.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020401-S000000-E235959.04.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020501-S000000-E235959.05.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020601-S000000-E235959.06.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020701-S000000-E235959.07.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020801-S000000-E235959.08.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20020901-S000000-E235959.09.V06B.HDF5
F:/17_Article/01_Data/11_precipitation\3B-MO.MS.MRG.3IMERG.20021001-S000000-E235959.10.V06B.HDF5
F:/17_Article/01_Da

In [4]:
def coordExtractionFromRaster(RasterName, GeoPandasDataFrame, NewColumnName):
    rasterFile = rasterio.open(RasterName)
    coords_extration = GeoPandasDataFrame.copy()
    rasterArray = rasterFile.read(1)
    
    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
        
    valueArray = np.array(valueArray)
    coords_extration[NewColumnName] = valueArray
    rasterFile = None
    return coords_extration

In [5]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")
precipitationMeanResult = Parallel(n_jobs=4)(delayed(coordExtractionFromRaster)(rawFileLocation + "\\temp\\" + str(int(year)) + "_precipitation_m.tif", coords_extration, 'annualPrecipitation') for year in np.linspace(2001, 2020, 20))

In [ ]:
def mergeParallelDf(DF):
    singleDF_0 = DF[0]
    singleDF_1 = DF[1]
    singleDF_2 = DF[2]
    singleDF_3 = DF[3]
    singleDF_4 = DF[4]
    singleDF_5 = DF[5]
    singleDF_6 = DF[6]
    singleDF_7 = DF[7]
    singleDF_8 = DF[8]
    singleDF_9 = DF[9]
    singleDF_10 = DF[10]
    singleDF_11 = DF[11]
    singleDF_12 = DF[12]
    singleDF_13 = DF[13]
    singleDF_14 = DF[14]
    singleDF_15 = DF[15]
    singleDF_16 = DF[16]
    singleDF_17 = DF[17]
    singleDF_18 = DF[18]
    singleDF_19 = DF[19]

    singleDF_0['year'] = 2001
    singleDF_1['year'] = 2002 
    singleDF_2['year'] = 2003 
    singleDF_3['year'] = 2004 
    singleDF_4['year'] = 2005 
    singleDF_5['year'] = 2006 
    singleDF_6['year'] = 2007 
    singleDF_7['year'] = 2008 
    singleDF_8['year'] = 2009 
    singleDF_9['year'] = 2010 
    singleDF_10['year'] = 2011
    singleDF_11['year'] = 2012 
    singleDF_12['year'] = 2013 
    singleDF_13['year'] = 2014 
    singleDF_14['year'] = 2015 
    singleDF_15['year'] = 2016 
    singleDF_16['year'] = 2017 
    singleDF_17['year'] = 2018 
    singleDF_18['year'] = 2019 
    singleDF_19['year'] = 2020

    mergeDF = pd.concat([
        singleDF_0, singleDF_1, singleDF_2, singleDF_3, singleDF_4, singleDF_5, singleDF_6, singleDF_7, singleDF_8, singleDF_9,
        singleDF_10, singleDF_11, singleDF_12, singleDF_13, singleDF_14, singleDF_15, singleDF_16, singleDF_17, singleDF_18, singleDF_19
    ])

    return mergeDF

In [ ]:
mergePrecipitationMeanResult = mergeParallelDf(precipitationMeanResult)
mergePrecipitationMeanResult = pd.DataFrame(mergePrecipitationMeanResult.drop(columns='geometry'))
mergePrecipitationMeanResult = mergePrecipitationMeanResult[['G04c_001', 'annualPrecipitation', 'year']]
mergePrecipitationMeanResult = mergePrecipitationMeanResult.set_index(['G04c_001', 'year'])

In [1]:
mergePrecipitationMeanResult [pd.isna(mergePrecipitationMeanResult['annualPrecipitation'])].shape

NameError: name 'mergePrecipitationMeanResult' is not defined

In [ ]:
mergePrecipitationMeanResult['annualPrecipitation'] = mergePrecipitationMeanResult['annualPrecipitation'].fillna(0)

In [ ]:
mergePrecipitationMeanResult [pd.isna(mergePrecipitationMeanResult['annualPrecipitation'])].shape

# Merge and Save Out
now includes longNtlResult and longNppResult

In [188]:
longFinalDf = pd.concat([longNtlDF, longNppDF], axis=1)

In [193]:
longFinalDf["NPP"] = longFinalDf["NPP"].fillna(0)

In [194]:
longFinalDf = pd.concat([longFinalDf, mergeTempResult], axis=1)

In [195]:
longFinalDf.head()

,,NTL,NPP,dayTimeMeanTemp,dayTimeStdTemp,nightTimeMeanTemp,nightTimeStdTemp
G04c_001,year,,,,,,
303660001,2001,0.0,0.0,0.0,0.0,0.0,0.0
303660002,2001,0.0,0.0,0.0,0.0,0.0,0.0
303660003,2001,0.0,0.0,0.0,0.0,0.0,0.0
303660004,2001,0.0,0.0,0.0,0.0,0.0,0.0
303660011,2001,0.0,0.0,0.0,0.0,0.0,0.0


In [196]:
longFinalDf.to_pickle("F:/17_Article/01_Data/98_20yearPickles/02_NtlNppPrecTemp.pkl")

In [180]:
#longFinalDf = pd.read_pickle("F:/17_Article/01_Data/98_20yearPickles/02_NtlNppPrecTemp.pkl") 

In [171]:
from IPython import get_ipython
get_ipython().magic('reset -sf')